Chapter 2.a.i. Lambda with a fixed number of parameters
=======

**Lambda** expressions and their reductions are critical components of the core deriviation system of **Prove-It**. A **Lambda** expression defines a function by mapping one or more parameters to an expression involving those parameters. For example $(x,y,z) \mapsto (x+y) \cdot z$ defines a function that operates on three parameters by adding the first two then multiplying that sum by the third. In lambda calculus terminology, this way of defining an function is called an *abstraction*.

In this chapter, we discuss **Lambda** expressions with a fixed number of parameters. We will discuss creation of Lambda expressions (abstraction), *relabeling* its parameters while retaining its meaning (alpha conversion), and *applying* the **Lambda** function to operators (beta reduction). The latter reduction rule plays an important role in proof deriviations in **Prove-It**. Specifically, the **instantiation** derivation rule is implemented via *lambda application*.

In the next chapter, we will discuss **Lambda** expressions with an indeterminate number of parameters.

In [1]:
import proveit
%begin lambda_reductions

## Creating Lambda expressions (abstraction)

Let's create our example expression after importing some necessary classes and objects.

In [2]:
from proveit import Lambda
from proveit import a, b, c, x, y, z
from proveit.numbers import Add, Mult
sum_then_mult = Lambda((x, y, z), Mult(Add(x, y), z))

A **Lambda** expression has `parameter(s)` and a `body`:

In [3]:
sum_then_mult.parameters

(x, y, z)

In [4]:
sum_then_mult.body

(x + y) * z

If there is a single parameter, just use `parameter`:

In [5]:
single_param_lambda = Lambda(x, Mult(x, x))

In [6]:
single_param_lambda.parameter

x

The body may be any type of expression. The parameter(s) must each be a variable, indexed variable, or range over indexed variables. We will discuss the latter possibilities later in this chapter.

In [7]:
try:
    Lambda((x, y, Add(x, y)), Mult(Add(x, y), z))
    assert False, "Expected a TypeError to be raised"
except TypeError as e:
    print("Expected Error:", e)

Expected Error: Parameter must be a Variable, Indexed variable, or range (ExprRange) over Indexed variables.  x + y fails to meet this requirement.


Also note that parameter variables must be unique.

In [8]:
from proveit import ParameterCollisionError
try:
    Lambda((x, x), Mult(Add(x, x), z))
    assert False, "Expected a ParameterCollisionError to be raised"
except ParameterCollisionError as e:
    print("Expected Error:", e)

Expected Error: Parameter variables must be unique.  (x, x) does not satisfy this criterion.


## Relabeling (alpha conversion)

The name of the parameters is irrelevant. The following is equivalent to the previous example: $(a,b,c) \mapsto (a+b)/c$. It uses different parameter labels but defines the same mapping. In **Prove-It**, we call this relabeling and these two **Lambda** expressions would be equal. In lambda calculus terminology, this is known as *alpha* conversion.

In [9]:
sum_then_mult # recall our earlier example

(x, y, z) -> ((x + y) * z)

In [10]:
sum_then_mult_relabeled = sum_then_mult.relabeled({x:a, y:b, z:c})

In [11]:
assert sum_then_mult_relabeled == sum_then_mult

We still have the original expression in its different form

In [12]:
sum_then_mult

(x, y, z) -> ((x + y) * z)

**Prove-It** has overloaded `Expression.__eq__` to regard these two objects as effectively equal and therefore interchangeable for the purposes of proof derivations. It does this by internally relabeling into a canonical form by choosing labels in a specific order according to the number of nested lambda expressions (similar, but different, to de Bruijn indices which labels variable occurrences according to the number of intervening lambda bindings):

In [13]:
sum_then_mult.canonically_labeled()

(_c, _b, _a) -> ((_c + _b) * _a)

The variables used for the canonical relabeling have preceeding underscores to distinguish them from user-specified variables.  We call these "dummy" variables.

These objects, the original expression and the canonical version, are not the same in all aspects. They have different representations. Having flexibility in ones choice of representation is important. In **Prove-It**, we call this *style*. One is free to manipulate the *style* of an expression while its *meaning* remains the same. Expressions with the same *meaning* will be equal via `==` and therefore interchangeable for the purposes of proof derivations. We will discuss more about *style* in a later chapter.

## Lambda application (beta reduction)

Applying the lambda function largely amounts to replacing the parameters as they appear in the lambda body with the provided operands:

In [14]:
sum_then_mult

(x, y, z) -> ((x + y) * z)

In [15]:
operands = [Add(a, x), Mult(b, y), Add(b, y, x)]

In [16]:
sum_then_mult.apply(*operands)

((a + x) + (b * y)) * (b + y + x)

The replacement is not always direct as it is in the above example. The reduction rules discussed in later chapters may be applied in the process.

Note that there is no type-checking in this process. The way **Prove-It** effects type-checking is via **Conditional** expressions discussed in a later chapter and via universal quantifier conditions when performing the **Instantiation** derivation rule. In these basic examples, anything goes. We can mix logical and numerical operations without any complaint:

In [17]:
from proveit.logic import And, Or
operands = [Or(a, x), And(b, y), Or(b, y, x)]

In [18]:
sum_then_mult.apply(*operands)

((a or x) + (b and y)) * (b or y or x)

### Automatic relabeling to avoid collisions

A lambda sub-expression defines a new scope for its parameters. This is clear from the fact that alpha conversion may be used to change its parameters arbitrarily without anything external to that sub-expression. Consider the following example

In [19]:
nested_lambda = Lambda(x, Lambda(y, Add(x, y)))

This defines a function that produces a function which produces the sum of the respective parameters. First, let's apply `nested_lambda` to something with no collissions with $y$:

In [20]:
nested_lambda.apply(Mult(a, b))

y -> ((a * b) + y)

We produced a lambda function in terms of the $y$ parameter. But what happens if we apply this `nested_lambda` to something that happens to be a function of $y$?

In [21]:
applied_nested_lambda = nested_lambda.apply(Mult(a, y))

The system automatically relabeled the $y$ of the nested lambda expression to a new variable to avoid the collision. It uses ${_{-}a}$ as the first available dummy variable, which has no relation to $a$ as a variable. (Note, if you happen to use the full alphabet, it will go on to ${_{-}aa}$, ${_{-}ab}$, etc. so it will never run out of options).

To understand that this is a correct and valid thing to do, consider that you could have relabeled $y$ to ${_{-}a}$ in the nested lambda expression first before calling `apply`. To relabel a lambda sub-expression, one can call `inner_expr()` to create an `InnerExpr` object. We will discuss this capability in more detail in Chapter 2.a.vi. but here is a quick prelude, relabeling the inner lambda expression.

In [22]:
relabeled_nested_lambda = \
    nested_lambda.inner_expr().body.relabeled({y:b})

And now there is clearly no conflict and we obtain the same result (but with a different lambda parameter):

In [23]:
applied_nested_lambda2 = relabeled_nested_lambda.apply(Mult(a, y))

These are equal expressions since they are the same up to alpha conversion.

In [24]:
applied_nested_lambda == applied_nested_lambda2

True

Lambda expressions are used implicitly in other types of expressions. They are used in `ExprRange` and `OperationOverInstances` type expressions. Examples of the later are $\forall, \exists, \sum, \prod$ operations. Automatic relabeling will happen in these cases as well, of course.

In [25]:
from proveit.logic import Exists, Equals
from proveit.numbers import zero
exists_fn = Lambda(x, Exists(y, Equals(Add(x, y), zero)))

In [26]:
exists_fn.apply(Mult(a, y))

exists_{_a} (((a * y) + _a) = 0)

### Masking parameters

For clarity reasons, it is generally best to avoid using the same parameter variable for a lambda expression within lambda expression. It is not disallowed, however. Importantly, scoping rules are obeyed such that the inner lambda expression masks any external uses of its parameter. For example, consider

In [27]:
masking_lambda = Lambda(a, And(a, Exists(a, Or(a, a))))

If we apply this `masking_lambda` function onto a general expression, the $a$ within the inner lambda expression (of the $\exists$ operation) will be left alone.

In [28]:
masking_lambda.apply(Or(x, y))

(x or y) and [exists_{a} (a or a)]

The `apply` method does have an optional `allow_relabeling` flag. When this is `True`, if it is valid to relabel an inner parameter with the same name as an external variable, it will do so. This is just a convenient way to perform alpha conversion in one step. It's purpose will be more apparent when we discuss **instantiation**. For example, turning `allow_relabeling` on, we get

In [29]:
masking_lambda.apply(b, allow_relabeling=True)

b and [exists_{b} (b or b)]

compared with the default of `allow_relabeling=False`

In [30]:
masking_lambda.apply(b)

b and [exists_{a} (a or a)]

But note that no relabeling occurs if it is not valid to do so.  Because relabeling is not allowed in the following example, `allow_relabeling=True` gives the same result as we had above with the `allow_relabeling=False` default:

In [31]:
masking_lambda.apply(Or(x, y), allow_relabeling=True)

(x or y) and [exists_{a} (a or a)]

In [32]:
%end lambda_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>